## Read in LOCA projections, filter to gridcells and time period, and obtain extreme values

August Posch --- February-March 2025

This notebook takes in downscaled gridded projections and a set of gridcells of interest, and returns extreme levels for two time periods of interest: midcentury (2031-2050) and end of century (2081-2100). Along the way it also saves the predicted daily precipitation time series and the associated annual precipitation max time series.

All necessary parameters are specified in the first code block, as a dictionary called `parameters`.

In [1]:
parameters = {
    'path_loca_2044': '../data/LOCA_downscaled/pr.CESM2-LENS.ssp370.r1i1p1f1.2015-2044.LOCA_16thdeg_v20240915.n_east.nc',
    'path_loca_2074': '../data/LOCA_downscaled/pr.CESM2-LENS.ssp370.r1i1p1f1.2045-2074.LOCA_16thdeg_v20240915.n_east.nc',
    'path_loca_2100': '../data/LOCA_downscaled/pr.CESM2-LENS.ssp370.r1i1p1f1.2075-2100.LOCA_16thdeg_v20240915.n_east.nc',
    'path_for_loading_gridcells_npy': '../data/auxil/loca_cells_of_interest_boston.npy',
    'save_daily_timeseries': True,
    'path_for_saving_daily': '../data/auxil/loca_daily_gba.csv',
    'save_annual_max_timeseries': True,
    'path_for_saving_annual_max': '../data/auxil/loca_annual_max_gba.csv'
}

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import os
from scipy.stats import genextreme as gev

In [3]:
ds1 = xr.open_dataset(parameters['path_loca_2044'], engine='netcdf4')
ds2 = xr.open_dataset(parameters['path_loca_2074'], engine='netcdf4')
ds3 = xr.open_dataset(parameters['path_loca_2100'], engine='netcdf4')

In [4]:
ds = xr.concat([ds1, ds2, ds3], 'time')

Steps:
1. Slice down to only the Boston gridcells as our restricted domain
2. Get annual max precip over that restricted domain (end up with time series, one number per year)
3. Use scipy GEV function to calculate 1-100 year value and 1-25 year value
4. Send those to Jack for flood modeling

Now load in those gridcells of interest.

In [5]:
cells = np.load(parameters['path_for_loading_gridcells_npy'])

## New March 12 - slice down to these gridcells in xarray.

Steps:
1. Create a new dataset with the coordinates of the old one. It should only have lat and lon dimensions. Call it `mask` and set every value to False.
2. Loop through my cells of interest, setting the value to True. Now `mask` is the proper mask.
3. convert ds to pandas dataframe only wherever `mask==True`.

In [6]:
only_latlons = ds.isel(time=0).drop('time')

In [7]:
mask = xr.full_like(only_latlons, False, dtype='bool')

In [8]:
for cell in cells:
    lon = cell[0]
    lat = cell[1]
    mask.pr.loc[lat,lon] = True

In [9]:
df_domain = ds.where(mask).to_dataframe().dropna()

Cool, df_domain only has data from our spatial domain of interest. Now group by day ("time") and aggregate (take the mean of) the precipitation ("pr") for each day.

Also note precip comes in kilograms per square meter per second. To get per day, we need 3600 seconds per hour and 24 hours per day. 3600*24 is 86400. But, then there's the issue of kilograms per square meter. With typical density of water, kilograms per square meter is the same as millimeters of precip.

In [10]:
areal_agg = df_domain.groupby('time').mean().reset_index()
areal_agg['prmm'] = areal_agg['pr'] * 86400

if parameters['save_daily_timeseries']:
    areal_agg[['time','prmm']].to_csv(parameters['path_for_saving_daily'])

In [11]:
areal_agg['year'] = areal_agg['time'].dt.year # add a year column

In [12]:
# yearly max df
ydf = areal_agg.groupby('year').max('prmm').reset_index()[['year','prmm']]

if parameters['save_annual_max_timeseries']:
    ydf.to_csv(parameters['path_for_saving_annual_max'])

Fit GEV distributions and get levels.

In [13]:
data = ydf.prmm[ydf.year.between(2031, 2050)]

shape, loc, scale = gev.fit(data)

print('Return levels for Boston, climate during 2031-2050, covering the watersheds where the subway is, in millimeters):')
print('1/100 year level:', gev.isf(1/100,shape,loc,scale))
print('1/25 year level:', gev.isf(1/25,shape,loc,scale))

Return levels for Boston, climate during 2031-2050, covering the watersheds where the subway is, in millimeters):
1/100 year level: 85.81062169587014
1/25 year level: 80.84303770162066


In [14]:
data = ydf.prmm[ydf.year.between(2081, 2100)]

shape, loc, scale = gev.fit(data)

print('Return levels for Boston, climate during 2081-2100, covering the watersheds where the subway is, in millimeters):')
print('1/100 year level:', gev.isf(1/100,shape,loc,scale))
print('1/25 year level:', gev.isf(1/25,shape,loc,scale))

Return levels for Boston, climate during 2081-2100, covering the watersheds where the subway is, in millimeters):
1/100 year level: 135.2133895888229
1/25 year level: 115.53216758246845
